In [1]:
#wandb
from kaggle_secrets import UserSecretsClient
import wandb
user_secrets = UserSecretsClient()

my_secret = user_secrets.get_secret("bert-rcnn-2") 

wandb.login(key=my_secret)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: storknguyen-286 (storknguyen-286-university-system-of-georgia) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
!gdown 18ZT8T7cUHvHNqoe_aZiDslR07lodEOK_
!unzip sentiment.zip
!rm sentiment.zip 
!ls

Downloading...
From (original): https://drive.google.com/uc?id=18ZT8T7cUHvHNqoe_aZiDslR07lodEOK_
From (redirected): https://drive.google.com/uc?id=18ZT8T7cUHvHNqoe_aZiDslR07lodEOK_&confirm=t&uuid=c55cbadc-25a2-4ad9-9a62-fe4a043b7679
To: /kaggle/working/sentiment.zip
100%|████████████████████████████████████████| 672M/672M [00:08<00:00, 79.4MB/s]
Archive:  sentiment.zip
  inflating: multi_cased_L-12_H-768_A-12/vocab.txt  
  inflating: multi_cased_L-12_H-768_A-12/bert_config.json  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: NTC_SV/NTC_SV_test.csv  
  inflating: NTC_SV/NTC_SV_train.csv  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
multi_cased_L-12_H-768_A-12  NTC_SV


In [3]:
!pip install transformers
!pip install torch
!pip install scikit-learn
!pip install numpy
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 5.7 MB/s eta 0:00:000:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-1

In [4]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd
from safetensors.torch import save_file
from tqdm import tqdm
import os

2025-04-26 10:06:02.708381: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745661962.895124      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745661962.951211      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [6]:
SEQ_LEN = 256
BATCH_SIZE = 16
EPOCHS = 10
LR = 2e-5
SAVE_EVERY = 5

In [7]:
train_df = pd.read_csv("/kaggle/working/NTC_SV/NTC_SV_train.csv").dropna()
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
le = LabelEncoder()
labels = le.fit_transform(train_df["label"])

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [8]:
class FoodyDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        enc = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )
        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "label": torch.tensor(self.labels[idx], dtype=torch.long)
        }

In [9]:
X_train, X_val, y_train, y_val = train_test_split(train_df['review'], labels, test_size=0.15, random_state=42)
train_dataset = FoodyDataset(X_train.tolist(), y_train, tokenizer, SEQ_LEN)
val_dataset = FoodyDataset(X_val.tolist(), y_val, tokenizer, SEQ_LEN)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

In [10]:
test_df = pd.read_csv("/kaggle/working/NTC_SV/NTC_SV_test.csv").dropna()
test_texts = test_df['review'].tolist()
test_labels = le.transform(test_df['label'])
test_dataset = FoodyDataset(test_texts, test_labels, tokenizer, SEQ_LEN)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [11]:
class BERT_RCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.bert = BertModel.from_pretrained("bert-base-multilingual-cased")
        self.conv1 = nn.Conv1d(768, 256, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.rnn = nn.LSTM(256, 128, num_layers=1, bidirectional=True, batch_first=True)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(128*2, num_classes)

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            bert_out = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state
        x = bert_out.permute(0, 2, 1)
        x = self.relu(self.conv1(x))
        x = x.permute(0, 2, 1)
        x, _ = self.rnn(x)
        x = torch.mean(x, dim=1)
        x = self.dropout(x)
        return self.fc(x)

In [12]:
model = BERT_RCNN(num_classes=len(set(labels))).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [13]:
os.makedirs("checkpoints", exist_ok=True)

In [14]:
print(device)

cuda


In [15]:
# 7. Huấn luyện
num_epochs = 10
save_every = 2
save_dir = "checkpoints"
os.makedirs(save_dir, exist_ok=True)

In [16]:
import wandb
import os
from tqdm import tqdm

# Khởi tạo W&B (chạy 1 lần đầu tiên sẽ yêu cầu đăng nhập)
wandb.init(project="my-project", config={
    "epochs": num_epochs,
    "batch_size": train_loader.batch_size,
    "learning_rate": optimizer.param_groups[0]['lr'],
    "model": "BERT + RCNN",
})

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct, total = 0, 0
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

    for batch in loop:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        loop.set_postfix(loss=loss.item())

        # 👉 Log loss từng batch (tuỳ chọn)
        wandb.log({"batch_loss": loss.item()})

    # 👉 Log loss & accuracy mỗi epoch
    epoch_loss = total_loss / len(train_loader)
    epoch_acc = correct / total
    wandb.log({
        "epoch": epoch + 1,
        "loss": epoch_loss,
        "accuracy": epoch_acc
    })

    print(f"\n✅ Epoch {epoch+1} — Loss: {epoch_loss:.4f} — Accuracy: {epoch_acc:.2%}")

    # 👉 Lưu mô hình định kỳ
    if (epoch + 1) % save_every == 0:
        ckpt_path = os.path.join(save_dir, f"bert_rcnn_epoch{epoch+1}.pth")
        torch.save(model.state_dict(), ckpt_path)
        print(f"💾 Đã lưu mô hình tại: {ckpt_path}")

        # 👉 Log checkpoint vào W&B
        wandb.save(ckpt_path)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch 1/10: 100%|██████████| 2166/2166 [05:22<00:00,  6.72it/s, loss=0.228]



✅ Epoch 1 — Loss: 0.4630 — Accuracy: 78.37%


Epoch 2/10: 100%|██████████| 2166/2166 [05:23<00:00,  6.69it/s, loss=0.281]



✅ Epoch 2 — Loss: 0.4135 — Accuracy: 81.36%
💾 Đã lưu mô hình tại: checkpoints/bert_rcnn_epoch2.pth


...


✅ Epoch 8 — Loss: 0.3725 — Accuracy: 83.48%
💾 Đã lưu mô hình tại: checkpoints/bert_rcnn_epoch8.pth


Epoch 9/10: 100%|██████████| 2166/2166 [05:22<00:00,  6.72it/s, loss=0.369] 



✅ Epoch 9 — Loss: 0.3667 — Accuracy: 83.70%


Epoch 10/10: 100%|██████████| 2166/2166 [05:21<00:00,  6.73it/s, loss=0.0777]



✅ Epoch 10 — Loss: 0.3606 — Accuracy: 84.09%
💾 Đã lưu mô hình tại: checkpoints/bert_rcnn_epoch10.pth


In [17]:
# Lưu tokenizer
save_dir_token_model = '/kaggle/working/token'
tokenizer.save_pretrained(save_dir_token_model)
print(f"Đã lưu tokenizer tại: {save_dir_token_model}")


Đã lưu tokenizer tại: /kaggle/working/token


In [18]:
!zip -r tokenizer_bert_rcnn.zip /kaggle/working/token

  adding: kaggle/working/token/ (stored 0%)
  adding: kaggle/working/token/tokenizer_config.json (deflated 75%)
  adding: kaggle/working/token/special_tokens_map.json (deflated 42%)
  adding: kaggle/working/token/vocab.txt (deflated 45%)


In [19]:
!pip install safetensors

In [20]:
from safetensors.torch import save_file
import os

# Lưu state_dict của model
state_dict = model.state_dict()

# Đường dẫn để lưu
save_path = os.path.join("/kaggle/working/", "model.safetensors")
save_file(state_dict, save_path)

print(f"✅ Đã lưu trọng số safetensors tại: {save_path}")


✅ Đã lưu trọng số safetensors tại: /kaggle/working/model.safetensors


In [21]:
model.eval()
preds, trues = [], []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids, attention_mask)
        pred = torch.argmax(outputs, dim=1)
        preds.extend(pred.cpu().numpy())
        trues.extend(labels.cpu().numpy())

print("=== Test Result ===")
print(classification_report(trues, preds))

=== Test Result ===
              precision    recall  f1-score   support

           0       0.87      0.83      0.85      5000
           1       0.84      0.88      0.86      5000

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000

